## 4.4 ListOfLists

Wykorzystamy `Jetpack Navigation` wraz z `RecyclerView` aby stworzyć prostą aplikację w modelu **master-detail**.

### **Fragmenty**

W aplikacji będziemy posiadać dwa fragmenty hostowane przez aktywność. Oba fragmenty będą zawierać `RecyclerView` - na jednym będzie znajdowała się lista liter, nac drugim lista słów na daną literę. Rozpocznijmy od dodania dwóch fragmentów.

In [ ]:
class LetterFragment : Fragment() {

    override fun onCreateView(
        inflater: LayoutInflater, container: ViewGroup?,
        savedInstanceState: Bundle?
    ): View? {
        return inflater.inflate(R.layout.fragment_letter, container, false)
    }
}

```xml
<FrameLayout xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:tools="http://schemas.android.com/tools"
    android:layout_width="match_parent"
    android:layout_height="match_parent"
    tools:context=".LetterFragment">

    <androidx.recyclerview.widget.RecyclerView
        android:id="@+id/letterRecyclerView"
        android:layout_width="match_parent"
        android:layout_height="match_parent"
        android:clipToPadding="false"
        android:padding="16dp" />

</FrameLayout>
```

In [ ]:
class WordFragment : Fragment() {

    override fun onCreateView(
        inflater: LayoutInflater, container: ViewGroup?,
        savedInstanceState: Bundle?
    ): View? {
        return inflater.inflate(R.layout.fragment_word, container, false)
    }
}

```xml
<FrameLayout xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:tools="http://schemas.android.com/tools"
    android:layout_width="match_parent"
    android:layout_height="match_parent"
    tools:context=".WordFragment">

    <androidx.recyclerview.widget.RecyclerView
        android:id="@+id/wordRecyclerView"
        android:layout_width="match_parent"
        android:layout_height="match_parent"
        android:clipToPadding="false"
        android:padding="16dp" />

</FrameLayout>
```

### **Adaptery**

W kolejnym kroku napiszemy adaptery dla liter i słów. Rozpocznijmy od layoutu który będą wykorzystywały oba adaptery (`item_view.xml`)

```xml
<?xml version="1.0" encoding="utf-8"?>
<Button xmlns:android="http://schemas.android.com/apk/res/android"
    android:id="@+id/button_item"
    android:layout_width="match_parent"
    android:layout_height="wrap_content"
    android:layout_margin="4dp"
    android:padding="8dp" />
```

Przejdźmy do `LetterAdapter`

In [ ]:
class LetterAdapter : RecyclerView.Adapter<LetterAdapter.LetterViewHolder>(){
    private val list = ('A').rangeTo('Z').toList()

    class LetterViewHolder(private val view: View) : RecyclerView.ViewHolder(view) {
        val button: Button = view.findViewById(R.id.button_item)
    }

    override fun getItemCount(): Int = list.size

    override fun onCreateViewHolder(parent: ViewGroup, viewType: Int): LetterViewHolder {
        return LetterViewHolder(LayoutInflater
            .from(parent.context)
            .inflate(R.layout.item_view, parent, false))
    }

    override fun onBindViewHolder(holder: LetterViewHolder, position: Int) {
        val item = list[position]
        holder.button.text = item.toString()
    }
}

`WordAdapter` będzie zawierał listę słów zaczynających się na literę, którą użytkownik wybierze na ekranie `FragmentLetter` - lista wszystkich śłów znajduje się w pliku `arrays.xml`

In [ ]:
class WordAdapter(private val letter: String, context: Context) :
    RecyclerView.Adapter<WordAdapter.WordViewHolder>() {}

W konstruktorze głównym `WordAdapter` przyjmuje literę, który będzie wykorzystywany do przefiltrowania listy wszystkich słów. Wczytywanie oraz filtrowanie wykonujemy w bloku `init`

In [ ]:
class WordAdapter(private val letterId: String, context: Context) :
    RecyclerView.Adapter<WordAdapter.WordViewHolder>() {

    private val words: List<String>

    init {
        words = context.resources.getStringArray(R.array.words).toList()
            .filter { it.startsWith(letterId, ignoreCase = true) }
    }

    class WordViewHolder(private val view: View) : RecyclerView.ViewHolder(view) {
        val button: Button = view.findViewById(R.id.button_item)
    }

    override fun getItemCount(): Int = words.size

    override fun onCreateViewHolder(parent: ViewGroup, viewType: Int): WordViewHolder {
        return WordViewHolder(LayoutInflater
            .from(parent.context)
            .inflate(R.layout.item_view, parent, false))
    }

    override fun onBindViewHolder(holder: WordViewHolder, position: Int) {
        val item = words[position]
        holder.button.text = item
    }
}

Dodajmy `RecyclerView` do `LetterFragment` oraz połączmy go z `LetterAdapter` -wykonujemy to w metodzie `onViewCreated`

In [ ]:
class LetterFragment : Fragment() {

    override fun onCreateView(
        inflater: LayoutInflater, container: ViewGroup?,
        savedInstanceState: Bundle?
    ): View? {
        return inflater.inflate(R.layout.fragment_letter, container, false)
    }

    override fun onViewCreated(view: View, savedInstanceState: Bundle?) {
        super.onViewCreated(view, savedInstanceState)
        val recyclerView: RecyclerView = view.findViewById(R.id.letterRecyclerView)
        recyclerView.layoutManager = LinearLayoutManager(context)
        recyclerView.adapter = LetterAdapter()
    }
}

### **Nawigacja**

Rozpoczniemy od dodania odpowiednich wpisów do plików `build.gradle`. Do `build.gradle(Module)` dodajemy

In [ ]:
plugins {
    id 'com.android.application'
    id 'org.jetbrains.kotlin.android'
    id 'androidx.navigation.safeargs.kotlin'
}

dependencies {

    def nav_version = "2.5.0"
    implementation "androidx.navigation:navigation-fragment:$nav_version"
    implementation "androidx.navigation:navigation-ui:$nav_version"
}

do pliku `build.gradle(Project)` przed blokiem `plugins` dodajemy

In [ ]:
buildscript {
    repositories {
        google()
    }
    dependencies {
        def nav_version = "2.5.0"
        classpath "androidx.navigation:navigation-safe-args-gradle-plugin:$nav_version"
    }
}

Rozwijamy menu kontekstowe na katalogu `res` i wybieramy **New -> Android Resource File**. Jako **Resource type** wybieramy `Navigation`, wpisujemy nazwę i przechodzimy dalej. Dodajemy nasze dwa fragmenty do grafu przez kliknięcie **New Destination**

<img src="https://media0.giphy.com/media/lpAzNFvKo2YYtS713w/giphy.gif?cid=790b7611aa030c5383793eabae2f506a034df4ec59a8020e&rid=giphy.gif&ct=g" width="350" />

Zdefiniujmy akcję przejścia z `LetterFragment` do `WordFragment`

<img src="https://media4.giphy.com/media/aEEE3YWxiOFwZIcKev/giphy.gif?cid=790b7611d69d9d1eb8ad6387ac07248ff1600f4a68b48059&rid=giphy.gif&ct=g" width="350" />

Będziemy przekazywać literę wybraną przez użytkownika w `LetterFragment`, więc dodajmy argument do fragmentu docelowego. W tym celu otwierzmy panel boczny **Attributes** i w zakładce **Arguments** dodajemy argument typu `String`

<img src="https://media3.giphy.com/media/L2HWTrUXkh9wnGtnef/giphy.gif?cid=790b7611f0c8db96023b41e464386f4db9cc3251c5bfb711&rid=giphy.gif&ct=g" width="350" />

Ustawmy `LetterFragment` jako fragment domowy - czyli wyświetlany jako pierwszy

<img src="https://media1.giphy.com/media/keYDRfiD1JQUXWc0PG/giphy.gif?cid=790b761175c970a92c810f41bb0dc3c45596a4ab7e16a983&rid=giphy.gif&ct=g" width="350" />

Pełny kod `navigation.xml`

```xml
<?xml version="1.0" encoding="utf-8"?>
<navigation xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:app="http://schemas.android.com/apk/res-auto"
    xmlns:tools="http://schemas.android.com/tools"
    android:id="@+id/navigation"
    app:startDestination="@id/letterFragment">

    <fragment
        android:id="@+id/letterFragment"
        android:name="pl.edu.uwr.pum.listoflistskotlin.LetterFragment"
        android:label="fragment_letter"
        tools:layout="@layout/fragment_letter" >
        <action
            android:id="@+id/action_letterFragment_to_wordFragment"
            app:destination="@id/wordFragment" />
    </fragment>
    <fragment
        android:id="@+id/wordFragment"
        android:name="pl.edu.uwr.pum.listoflistskotlin.WordFragment"
        android:label="fragment_word"
        tools:layout="@layout/fragment_word" >
        <argument
            android:name="letter"
            app:argType="string" />
    </fragment>
</navigation>
```

Następnie dodajmy akcję do `LetterAdapter`

In [ ]:
override fun onBindViewHolder(holder: LetterViewHolder, position: Int) {
    val item = list[position]
    holder.button.text = item.toString()
    holder.button.setOnClickListener {
        val action = LetterFragmentDirections
            .actionLetterFragmentToWordFragment(
                letter = holder.button.text.toString()
            )
        holder.view.findNavController().navigate(action)
    }
}

Zmodyfikujmy layout głównej aktywności

```xml
<?xml version="1.0" encoding="utf-8"?>
<FrameLayout xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:tools="http://schemas.android.com/tools"
    xmlns:app="http://schemas.android.com/apk/res-auto"
    android:layout_width="match_parent"
    android:layout_height="match_parent"
    tools:context=".MainActivity">

    <androidx.fragment.app.FragmentContainerView
        android:id="@+id/nav_host_fragment"
        android:name="androidx.navigation.fragment.NavHostFragment"
        android:layout_width="match_parent"
        android:layout_height="match_parent"
        app:defaultNavHost="true"
        app:navGraph="@navigation/navigation"/>

</FrameLayout>
```

Przekazany argument odbieramy w `WordFragment` w metodzie `onCreate`

In [ ]:
class WordFragment : Fragment() {

    private lateinit var letter: String

    override fun onCreate(savedInstanceState: Bundle?) {
        super.onCreate(savedInstanceState)

        arguments?.let { letter = it.getString("letter").toString() }
    }

    override fun onCreateView(
        inflater: LayoutInflater, container: ViewGroup?,
        savedInstanceState: Bundle?
    ): View? {
        return inflater.inflate(R.layout.fragment_word, container, false)
    }

    override fun onViewCreated(view: View, savedInstanceState: Bundle?) {
        val recyclerView = view.findViewById<RecyclerView>(R.id.wordRecyclerView)
        recyclerView.layoutManager = LinearLayoutManager(requireContext())
        recyclerView.adapter = WordAdapter(letter, requireContext())
    }
}

Zwróćmy uwagę na klucz "letter" - jest to nazwa ustalona przy dodawaniu argumentu w nawigacji

```xml
    <fragment
        android:id="@+id/wordFragment"
        android:name="pl.edu.uwr.pum.listoflistskotlin.WordFragment"
        android:label="fragment_word"
        tools:layout="@layout/fragment_word" >
        <argument
            android:name="letter"
            app:argType="string" />
    </fragment>
```

### **Implicit Intent**

Ostatnim elementem w tej aplikacji będzie wykorzystanie mechanizmu domniemanych intentów w celu wyszukania przez wyszukiwarkę znaczenia danego słowa. Zrobimy to w `WordAdapter`

In [ ]:
override fun onBindViewHolder(holder: WordViewHolder, position: Int) {
    val item = words[position]
    holder.button.text = item

    holder.button.setOnClickListener {
        val queryUrl: Uri = Uri.parse("https://www.google.com/search?q=${item}")
        val intent = Intent(Intent.ACTION_VIEW, queryUrl)
        context.startActivity(intent)
    }
}

Możemy przetestować aplikację

<img src="https://media4.giphy.com/media/12aNUkEltsj9IJX3DK/giphy.gif?cid=790b761153d67ec7afbe0798454db3f988f74125c2830ad8&rid=giphy.gif&ct=g" width="150" />

Lista słów z pliku `arrays.xml`:

```xml
<?xml version="1.0" encoding="utf-8"?>
<!--
     Copyright (C) 2020 The Android Open Source Project

     Licensed under the Apache License, Version 2.0 (the "License");
     you may not use this file except in compliance with the License.
     You may obtain a copy of the License at

          http://www.apache.org/licenses/LICENSE-2.0

     Unless required by applicable law or agreed to in writing, software
     distributed under the License is distributed on an "AS IS" BASIS,
     WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
     See the License for the specific language governing permissions and
     limitations under the License.
-->
<resources>
    <string-array name="words">
        <item>aargh</item>
        <item>about</item>
        <item>acidic</item>
        <item>acne</item>
        <item>acrid</item>
        <item>acute</item>
        <item>alphabet</item>
        <item>anchor</item>
        <item>anecdote</item>
        <item>animal</item>
        <item>automobile</item>
        <item>awesome</item>
        <item>back</item>
        <item>balloon</item>
        <item>basket</item>
        <item>bench</item>
        <item>best</item>
        <item>birthday</item>
        <item>books</item>
        <item>brief</item>
        <item>business</item>
        <item>buy</item>
        <item>camera</item>
        <item>catapult</item>
        <item>center</item>
        <item>children</item>
        <item>city</item>
        <item>class</item>
        <item>click</item>
        <item>coffee</item>
        <item>comment</item>
        <item>company</item>
        <item>contact</item>
        <item>copyright</item>
        <item>creative</item>
        <item>data</item>
        <item>day</item>
        <item>details</item>
        <item>development</item>
        <item>dine</item>
        <item>dream</item>
        <item>dusk</item>
        <item>each</item>
        <item>education</item>
        <item>electric</item>
        <item>elephant</item>
        <item>email</item>
        <item>emerald</item>
        <item>event</item>
        <item>every</item>
        <item>family</item>
        <item>file</item>
        <item>find</item>
        <item>first</item>
        <item>flair</item>
        <item>flight</item>
        <item>follow</item>
        <item>forum</item>
        <item>fox</item>
        <item>free</item>
        <item>frequent</item>
        <item>full</item>
        <item>game</item>
        <item>general</item>
        <item>glass</item>
        <item>good</item>
        <item>gram</item>
        <item>gray</item>
        <item>great</item>
        <item>green</item>
        <item>group</item>
        <item>guide</item>
        <item>guitar</item>
        <item>haircut</item>
        <item>health</item>
        <item>hearsay</item>
        <item>help</item>
        <item>here</item>
        <item>home</item>
        <item>honey</item>
        <item>hundred</item>
        <item>hurried</item>
        <item>ice</item>
        <item>icon</item>
        <item>inbox</item>
        <item>incomplete</item>
        <item>information</item>
        <item>investment</item>
        <item>item</item>
        <item>jackal</item>
        <item>january</item>
        <item>job</item>
        <item>join</item>
        <item>joke</item>
        <item>journal</item>
        <item>july</item>
        <item>jump</item>
        <item>june</item>
        <item>kangaroo</item>
        <item>keep</item>
        <item>keyboard</item>
        <item>kind</item>
        <item>kingdom</item>
        <item>knowledge</item>
        <item>known</item>
        <item>koala</item>
        <item>landscape</item>
        <item>last</item>
        <item>late</item>
        <item>laugh</item>
        <item>lemon</item>
        <item>letter</item>
        <item>lily</item>
        <item>link</item>
        <item>list</item>
        <item>local</item>
        <item>long</item>
        <item>madeleine</item>
        <item>magazine</item>
        <item>make</item>
        <item>many</item>
        <item>marine</item>
        <item>maze</item>
        <item>meditate</item>
        <item>message</item>
        <item>momentum</item>
        <item>moon</item>
        <item>more</item>
        <item>most</item>
        <item>mountain</item>
        <item>music</item>
        <item>name</item>
        <item>need</item>
        <item>negotiate</item>
        <item>news</item>
        <item>next</item>
        <item>night</item>
        <item>north</item>
        <item>not</item>
        <item>now</item>
        <item>number</item>
        <item>number</item>
        <item>oak</item>
        <item>octopus</item>
        <item>online</item>
        <item>only</item>
        <item>opposite</item>
        <item>order</item>
        <item>other</item>
        <item>pack</item>
        <item>page</item>
        <item>painting</item>
        <item>people</item>
        <item>pillow</item>
        <item>pizza</item>
        <item>podcast</item>
        <item>polar</item>
        <item>policy</item>
        <item>post</item>
        <item>presentation</item>
        <item>price</item>
        <item>product</item>
        <item>public</item>
        <item>puppy</item>
        <item>puzzle</item>
        <item>quality</item>
        <item>quantity</item>
        <item>query</item>
        <item>quest</item>
        <item>question</item>
        <item>quick</item>
        <item>quince</item>
        <item>quite</item>
        <item>quotation</item>
        <item>read</item>
        <item>real</item>
        <item>recipe</item>
        <item>report</item>
        <item>research</item>
        <item>restaurant</item>
        <item>review</item>
        <item>revolve</item>
        <item>rewind</item>
        <item>right</item>
        <item>search</item>
        <item>secret</item>
        <item>see</item>
        <item>seed</item>
        <item>service</item>
        <item>ship</item>
        <item>should</item>
        <item>site</item>
        <item>skill</item>
        <item>stargaze</item>
        <item>state</item>
        <item>style</item>
        <item>sunrise</item>
        <item>taxi</item>
        <item>tidy</item>
        <item>time</item>
        <item>together</item>
        <item>tourist</item>
        <item>trailer</item>
        <item>travel</item>
        <item>truck</item>
        <item>under</item>
        <item>unicorn</item>
        <item>uniform</item>
        <item>unique</item>
        <item>united</item>
        <item>university</item>
        <item>uplife</item>
        <item>useful</item>
        <item>value</item>
        <item>various</item>
        <item>vase</item>
        <item>version</item>
        <item>very</item>
        <item>video</item>
        <item>view</item>
        <item>violin</item>
        <item>vision</item>
        <item>visit</item>
        <item>volume</item>
        <item>walrus</item>
        <item>wander</item>
        <item>well</item>
        <item>whirlwind</item>
        <item>winter</item>
        <item>world</item>
        <item>would</item>
        <item>xanthin</item>
        <item>xenial</item>
        <item>xenolith</item>
        <item>xenon</item>
        <item>xiphoid</item>
        <item>yacht</item>
        <item>year</item>
        <item>yield</item>
        <item>yoga</item>
        <item>yogurt</item>
        <item>young</item>
        <item>your</item>
        <item>zein</item>
        <item>zero</item>
        <item>zigzag</item>
        <item>zine</item>
        <item>zone</item>
        <item>zoo</item>
        <item>zouk</item>
    </string-array>
</resources>
```